In [ ]:
from dlsite_analyzer.config import RAW_JSON_DATA_DIR
from dlsite_analyzer import (
    initialize_database,
    fetch_and_save_voice_works,
    import_voice_works_to_db,
)

In [2]:
# データベースにテーブルが存在しない場合は初期化する
initialize_database()

In [ ]:
# ボイス作品のデータをJSONファイルに保存
fetch_and_save_voice_works(RAW_JSON_DATA_DIR)

2024-11-08 19:00:31 INFO     dlsite_analyzer Total pages to process: 545
100%|██████████| 545/545 [43:58<00:00,  4.84s/it]
2024-11-08 19:44:30 INFO     dlsite_analyzer Finished saving voice works to JSON files.
